# Transformer architectures for audio

## Refresher on transformer models

The original transformer architecture was introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017. It consists of an encoder-decoder structure where both the encoder and decoder are composed of multiple layers of self-attention mechanisms and feed-forward neural networks.


$$
\mathrm{Attention}(Q, K, V)
\;=\;
\mathrm{softmax}\!\Bigl(\frac{Q\,K^{\top}}{\sqrt{d_k}}\Bigr)\;V
$$
For multi-head attention, the query (Q), key (K), and value (V) matrices are split into multiple heads, allowing the model to jointly attend to information from different representation subspaces at different positions. The multi-head attention mechanism can be expressed as follows:

$$
\mathrm{MultiHead}(Q, K, V)
\;=\;
\mathrm{Concat}\bigl(\mathrm{head}_1,\dots,\mathrm{head}_h\bigr)\;W^O,
\quad
$$

$$
\text{where }
\mathrm{head}_i
=
\mathrm{Attention}\bigl(QW_i^Q,\;K\,W_i^K,\;V\,W_i^V\bigr).
$$


### Encoder - Decoder

The **Encoder** receives an input sequence of tokens (embeddings of words or images with positional encodings). This part tries to acquire understanding of the input. This is done by computing attention scores between all pairs of tokens in the sequence and using these scores to create a weighted representation of the input. Using skip connections and layer normalization, the model can learn complex relationships between tokens and can capture long-range dependencies.


The **Decoder** uses the encoder's output to generate a sequence of tokens. It also uses self-attention to attend to the previously generated tokens, allowing it to generate sequences in an autoregressive manner. The decoder's attention mechanism is masked to prevent attending to future tokens, ensuring that the model generates sequences in a left-to-right manner.

There are also transformer-based models that only use the encoder part (good for tasks that require understanding of the input, such as classification), or only the decoder part (good for tasks such as text generation). An example of an encoder-only model is BERT; an example of a decoder-only model is GPT2.

### Inputs and outputs

For audio tasks, the input and/or output sequences may be audio instead of text:
- Automatic speech recognition (ASR): The input is speech, the output is text.
- Speech synthesis (TTS): The input is text, the output is speech.
- Audio classification: The input is audio, the output is a class probability — one for each element in the sequence or a single class probability for the entire sequence.
- Voice conversion or speech enhancement: Both the input and output are audio.

There are a few different ways to handle audio so it can be used with a transformer. The main consideration is whether to use the audio in its raw form — as a waveform — or to process it as a spectrogram instead.

One downside of using the raw waveform as input is that they tend to have long sequence lengths. For example, thirty seconds of audio at a sampling rate of 16 kHz gives an input of length 30 * 16000 = 480000. Longer sequence lengths require more computations in the transformer model, and so higher memory usage.

Because of this, raw audio waveforms are not usually the most efficient form of representing an audio input. By using a spectrogram, we get the same amount of information but in a more compressed form.

The log-mel spectrogram is then processed by a small CNN into a sequence of embeddings, which goes into the transformer as usual.

### Model outputs




## Connectionist Temporal Classification (CTC) Architecture


CTC is a technique used with encoder only transformers for automatic speech recognition (ASR) tasks. It allows the model to output a sequence of labels (e.g., characters or words) without requiring explicit alignment between the input audio and the output text.

**TLDR**:
The model predicts one token (character) for every 20 ms of (partially overlapping) audio from the input waveform. This gives a lot of duplicates. Thanks to the CTC blank token, we can easily remove these duplicates without destroying the proper spelling of the words. This is a very simple and convenient way to solve the problem of aligning the output text with the input audio.


The encoder reads the input sequence (the audio waveform) and maps this into a sequence of hidden-states, also known as the output embeddings.


With a CTC model, we apply an additional linear mapping on the sequence of hidden-states to get class label predictions. The class labels are the **characters of the alphabet** (a, b, c, …). This way we’re able to predict any word in the target language with a small classification head, as the vocabulary just needs to exist of 26 characters plus a few special tokens.


Here’s the rub: In speech, we don’t know the alignment of the audio inputs and text outputs. We know that the order the speech is spoken in is the same as the order that the text is transcribed in (the alignment is so-called monotonic), but we don’t know how the characters in the transcription line up to the audio. This is where the CTC algorithm comes in.

The CTC downsamples the audio input sequence to a smaller sequence of hidden-states. This is done by applying a convolutional neural network (CNN) to the input audio waveform, which reduces the sequence length of 25 ms to 768 hidden-states (with 5 ms being overlap).

$$
50\text{ Hidden states} = 1 \text{ seconds of audio} / 0.020 \text{ time per hidden-state}
$$

For one second of audio, we then forward a sequence of 50 hidden-states to the transformer encoder. (The audio segments extracted from the input sequence partially overlap, so even though one hidden-state vector is emitted every 20 ms, each hidden-state actually represent 25 ms of audio.)

To make the text predictions we map each of the 768 hidden-states to a character label using a linear layer. This gives us a sequence of character predictions, one for each hidden-state.

(50, 32) where 50 is the number of hidden-states and 32 is the number of characters in the alphabet (including special tokens).

## Sequence to Sequence (Seq2Seq) Architecture
The Seq2seq models maps a sequence of one kind of data to a sequence of another kind of data

They are autoregressive models that generate output sequences one token at a time, conditioning on the input sequence and the previously generated tokens. 
In general is Sequence to Sequence better then CTC even when the CTC is combined with an external language model. When using the same training data with the same loss function, the Seq2seq model outperforms CTC giving greater flexibility and generally superior performance.

Seq2seq models often use Cross-Entropy loss to train the model. The model is trained to predict the next token in the output sequence given the input sequence and the previous tokens in the output sequence. This is done by using teacher forcing, where the model is provided with the correct output tokens during training.

## Audio Classification Architecture
One of the easiest ways to perform audio classification is to pretend it’s an image classification problem!

We take the spectrogram of the audio that is a 2D tensor of shape (frequencies, sequence length) and we pass the image into a regular CNN classifier model such as ResNet, EfficientNet, Inception, etc. The model will learn to classify the audio based on the patterns in the spectrogram. One can even pass them into a ViT model, which is a transformer-based model for image classification.


This is what Audi Spectrogram Transformer (AST) does. It uses the ViT with the spectrogram as input. Due to the attention mechanism the model can capture long-range dependencies in the audio signal better then CNN's, which is important for audio classification tasks.

AST splits the audio spectrogram into a sequence of partially overlapping image patches 16x16 pixels and adds the positional encoding to the patches. The patches are then passed through the transformer encoder blocks. At the top of the encoder a linear layer is applied to the output of the transformer to get the class probabilities.

Important to note is that even though we pretend that audio is an image, data augmentation techniques that are used for images, such as random cropping, flipping, and rotation, are not suitable for audio data. 

